# Monte Carlo Markov Chains para Galaxy Clustering

Agora que vimos os conceitos básicos de MCMC e análise estatística, é a sua vez. A Gabriela gerou dados observacionais da densidade de galáxias, enquanto o Guilherme explicou como fornecer previsões teóricas. O objetivo é obter **intervalos de confiança** e gerar um ***corner plot*** para dois parâmetros cosmológicos:
- $\Omega_m$: fração que matéria não-relativística ("baryons" + dark matter + neutrinos massivos) representam da energia total no Universo;
- $\sigma_8$: variância do campo de densidade de matéria $\delta_m(\mathbf{x}, z = 0)$ dentro de esferas de raio $R = 8h/\mathrm{Mpc}$.

Vou deixar vocês com um código base, copiado da Gabriela e do Guilherme.

Me chamem se tiverem qualquer dúvida ou problema!

Dicas:
- O notebook `MCMC_supernovas.ipynb` já tem uma implementação de Metropolis-Hastings pronta. Você pode copiar e colar, mas tem que refatorar o código para esse problema:
  - É necessário mudar os parâmetros que são sampleados
  - Repensar priors e proposal (proposal Gaussiano funciona melhor, mas precisa de uma covariância)
- Os $C_\ell$ teóricos são dados em $\ell$ inteiro, enquanto os dados são binados, e portanto tem $\ell$ fracionário. Talvez você queira aplicar uma interpolação
- Você pode explorar alguns valores de $\Omega_m$ e $\sigma_8$ na mão usando a célula acima: isso pode te dar uma informação valiosa sobre o ponto inicial da MCMC, uma vez que começar de um ponto com alta *likelihood* melhora a velocidade de convergência

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from classy import Class

In [ ]:
zz, nz = np.load("data/dndz_input_z3_galaxy_field_lsstlike.npy")
np.savetxt("data/dndz_clustering_class.txt", np.stack((zz, nz), axis=1)) # Salvando no formato do CLASS

In [ ]:
def get_cl_gg(Omega_m, sigma_8, bias=1.5):
    lmax = 2200
    params = {
        # Saídas necessárias (CMB + LSS)
        'modes': 's',
        'output': 'tCl, pCl, lCl, mPk, nCl, dCl',
        'lensing': 'yes',
        'l_max_scalars': lmax,
        'l_max_lss': lmax,
        'number_count_contributions': 'density',

        # Energia escura via fld
        'Omega_Lambda': 0,
        'w0_fld': '-1.',
        'wa_fld': '0.0',

        # Espectro primordial (n_s fixo; sigma8 varia no loop)
        'n_s': 0.96,

        # Fundo
        'h': 0.673,
        'Omega_b': 0.03,

        # Verbosidade e gauge
        'background_verbose': 0,
        'perturbations_verbose': 0,
        'gauge': 'Synchronous',

        # P(k)
        'z_pk': '5.0, 0.0',
        'P_k_max_h/Mpc': 10,
        'k_per_decade_for_pk': 30,
        'dNdz_selection': 'data/dndz_clustering_class.txt',
    }
    Omega_nu_fid = 0.06/93.15/params['h']**2
    params.update({
        'Omega_cdm': Omega_m - params['Omega_b'] - Omega_nu_fid,
        'sigma8': sigma_8,
    })
    
    cosmo = Class()
    cosmo.set(params)
    cosmo.compute()
    
    cl_dens = cosmo.density_cl(lmax)
    Cl_gg = bias**2 * cl_dens['dd']['dens[1]-dens[1]']
    ell = np.arange(len(Cl_gg))
    return ell, np.asarray(Cl_gg)

In [ ]:
# Solução
from scipy.interpolate import interp1d

ell_data, cl_gg_data, sigma_cl_gg = np.load("teste_ell_Cl_errobar_gg.npy")

# Alguns pontos são zero, vamos retirá-los nos dados
mask = cl_gg_data > 0
ell_data = ell_data[mask]
cl_gg_data = cl_gg_data[mask]
sigma_cl_gg = sigma_cl_gg[mask]

# Exemplo de cosmologia vs teoria
ell_theory, cl_gg_theory = get_cl_gg(Omega_m=0.3, sigma_8=0.800)
plt.plot(ell_theory, cl_gg_theory)
plt.errorbar(ell_data, cl_gg_data, yerr=sigma_cl_gg, color="black", markersize=10, ls="none")

In [ ]:
ell_theory, cl_gg_theory = get_cl_gg(Omega_m=0.3, sigma_8=0.840)
cl_theory_interpolator = interp1d(ell_theory, cl_gg_theory)
cl_gg_theory = cl_theory_interpolator(ell_data)
plt.errorbar(ell_data, (cl_gg_data-cl_gg_theory)/cl_gg_theory, yerr=sigma_cl_gg/cl_gg_theory, color="black", markersize=20)

In [ ]:
# Solução:
def chi2(Omega_m, sigma_8):
    ell_theory, cl_gg_theory = get_cl_gg(Omega_m, sigma_8)
    cl_theory_interpolator = interp1d(ell_theory, cl_gg_theory)
    cl_gg_theory = cl_theory_interpolator(ell_data)
    delta = cl_gg_theory - cl_gg_data
    return np.sum(delta**2/sigma_cl_gg**2)

chi2(0.3, 0.800)

In [ ]:
from random import uniform
from time import time
class MCMCWalker:
    """
        Helper class for managing MCMCs. The class contains methods for performing Monte Carlo steps and saves the state.
    """
    def __init__(self):
        # Hard-coding an initial point based on the exploration
        initial_om = ...
        initial_sigma8 = ...
        initial_params = [initial_om, initial_sigma8]
        initial_chi2 = chi2(*initial_params)
        initial_sample = {
            'params': initial_params,
            'chi2': initial_chi2,
            'weight': 1,
        }
        self.samples = [initial_sample]

    def accept_sample(self, params, chi2):
        sample = {
            'params': params,
            'chi2': chi2,
            'weight': 1
        }
        self.samples.append(sample)

    def step(self):
        while True:
            current_chi2 = self.samples[-1]['chi2']
            step = np.random.multivariate_normal(np.zeros(2), cov)
            new_om = self.samples[-1]['params'][0] + step[0]
            new_s8 = self.samples[-1]['params'][1] + step[1]
            
            new_params = [new_om, new_s8]
            if new_om < 0 or new_om > 1 or new_s8 > 1.5 or new_s8 < 0.4:
                # Reject point outside the prior
                self.samples[-1]['weight'] += 1
                continue
            new_chi2 = chi2(*new_params)
            if new_chi2 == np.nan:
                # Reject points that have problematic chi2
                self.samples[-1]['weight'] += 1
                continue
            if new_chi2 < current_chi2:
                self.accept_sample(new_params, new_chi2)
                break
            else:
                r = uniform(0, 1)
                if r < np.exp(-(new_chi2 - current_chi2)/2):
                    self.accept_sample(new_params, new_chi2)
                    break
                else:
                    self.samples[-1]['weight'] += 1 # Increment weight
                    continue
            
    
    def gelman_rubin(self, n_split):
        all_params = np.array(
            [sample['params'] for sample in self.samples]
        )[:-(len(self.samples)%n_split)]
        np.random.shuffle(all_params)
        split_params = np.split(all_params, n_split)
        avg = np.mean(split_params, axis=1)
        std = np.std(split_params, axis=1)
        avg_of_std = np.mean(std, axis=0)
        std_of_avg = np.std(avg, axis=0)
        R_minus_one = std_of_avg/avg_of_std
        return np.max(R_minus_one)

def run_mcmc(w):
    print("Starting MCMC")
    start = time()
    while True:
        for _ in range(10): w.step()
        R_minus_one = w.gelman_rubin(4)
        print(f"At {len(w.samples)} samples, R-1 = {R_minus_one}")
        if R_minus_one < 0.025: break 
    print(f"MCMC Converged! Took {time() - start:.2f} seconds")

In [ ]:
w = MCMCWalker()

In [ ]:
run_mcmc(w)

In [ ]:
import getdist
from getdist import plots

def getdist_chain(w):
    """
        Função auxiliar que transforma as amostras cruas em um objeto `getdist.MCSamples`
        A função também define o parâmetro derivado `S8`
    """
    mcmc = getdist.MCSamples(
        samples=np.array([sample['params'] + [sample['chi2']] for sample in w.samples]),
        weights=np.array([sample['weight'] for sample in w.samples]),
        names=["Omega_m", "sigma_8", "chi2"],
        labels=["\\Omega_m", "\\sigma_8", "\\chi^2"],
        ranges={"Omega_m": (0, None)}
    )
    mcmc.removeBurn(0.15)
    mcmc.addDerived(mcmc["sigma_8"]*np.sqrt(mcmc["Omega_m"]/0.3), name="S8", label="S_8")
    return mcmc

chain = getdist_chain(w)

In [ ]:
# Obter intervalos de confiança 1D
params = ["Omega_m", "sigma_8", "S8"]
print("1D confidence intervals (68%):")
for param in params:
  print(chain.getInlineLatex(param, limit=1))

In [ ]:
# Obtendo contornos de confiança 2D, o corner plot
p = getdist.plots.get_subplot_plotter()
p.settings.axes_fontsize=22
p.settings.axes_labelsize=22
p.triangle_plot(
    chain,
    params=params,
    filled=True,
    contour_colors=["#406421"]
)
p.export("mcmc_gg.pdf")